In [16]:
import csv
import regex as re

import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from helpers import create_category_map, analyze_and_create_output_csv, create_dataframe, analyze_text_append_result
import os
import pandas as pd
import spacy
import textdescriptives as td
from timeit import default_timer as timer


In [29]:
lex = pd.read_csv('formas.totalbr.csv', encoding='iso-8859-1', sep='\t',quoting=csv.QUOTE_NONE)
#lex = lex.filter(regex=r'\d', axis=0)
#lex = lex[lex['palavra'].str.contains('\d', na=False)==False]
lex = lex[lex['palavra'].str.contains('^[A-Za-záàâãéèêíóôõúçÁÀÂÃÉÈÍÓÔÕÚÇ]+$', na=False)]
lex = lex[lex['freq'] >=0]
lex.dropna(inplace=True)
#lex.dropna(inplace=True)
#lex.sort_values('palavra', ascending=True, inplace=True)
lex

,freq,palavra
27,399539,A
38778,99,AA
90317,24,AAA
702517,1,AAAAAAHHHHH
702511,1,AAADT
...,...,...
429153,1,útlima
412798,1,úvula
407585,1,úzo
513436,1,úúnico


In [19]:
#lex[lex['palavra'].str.contains('^maluca$', na=False)]
1/lex.iloc[lex['freq'].idxmax()-1]['freq']

3.1916761087084884e-06

In [20]:
category_map = {'2_Ensino_Fundamental_II': 1,
 '3_Ensino_Medio': 2,
 '1_Ensino_Fundamental_I': 0,
 '4_Ensino_Superior': 3}

df = pd.read_csv('output3.csv')
df['category'] = df['category'].map(category_map)
df.astype({'category': 'category'}).dtypes

text                      object
passed_quality_check        bool
n_stop_words             float64
alpha_ratio              float64
mean_word_length         float64
                          ...   
entropy                  float64
perplexity               float64
per_word_perplexity      float64
category                category
name                      object
Length: 72, dtype: object

In [21]:
nlp = spacy.load('pt_core_news_sm')
text = df.iloc[0]['text']
doc = nlp(text)

In [22]:
text

'Como funciona, Origem das coisas. “É tarde! É tarde! É tarde, é tarde, é tarde!” O Coelho Branco, personagem do livro Alice no País das Maravilhas, do inglês Lewis Carroll (1832-1898), entrava em pânico toda vez que olhava o relógio. Seu desespero era um símbolo do que esses objetos, recém-incorporados ao cotidiano, estavam fazendo com a cabeça dos nossos bisavós, no século 19. Mais de 100 anos depois, é impossível pensar a existência sem o tique-taque. Ele assinala a hora de acordar, de dormir, de comer – e todas as demais atividades humanas. A necessidade de uma maneira única de medir o tempo deu origem aos primeiros relógios de sol, no Egito, há cerca de 5000 anos. Os aparelhos mecânicos, com ponteiros, surgiram na China, no século 8. Na Europa, começaram a ser usados por volta do ano 1000, quando os monges católicos decidiram que as atividades sérias, como a oração, deveriam ter hora para começar e para acabar. Três séculos mais tarde, apareceram os relógios das igrejas, que anunc

In [23]:

for token in doc:
    print(token, token.lemma, token.lemma_)

Como 3817173063683202752 como
funciona 15914192142941099050 funcionar
, 2593208677638477497 ,
Origem 11808335588263673568 Origem
das 12950005104698871082 de o
coisas 4777076820804454235 coisa
. 12646065887601541794 .
“ 16073121960489476521 “
É 16256712947286012828 é
tarde 2987168054197546619 tarde
! 17494803046312582752 !
É 16256712947286012828 é
tarde 2987168054197546619 tarde
! 17494803046312582752 !
É 940341820454552972 ser
tarde 2987168054197546619 tarde
, 2593208677638477497 ,
é 940341820454552972 ser
tarde 2987168054197546619 tarde
, 2593208677638477497 ,
é 940341820454552972 ser
tarde 2987168054197546619 tarde
! 17494803046312582752 !
” 6756288415164046634 ”
O 1489474827855109852 o
Coelho 17978853074236135148 Coelho
Branco 4385255702263241246 Branco
, 2593208677638477497 ,
personagem 14412324136959110456 personagem
do 12950005104698871082 de o
livro 2922908963672587675 livro
Alice 11143966581707015846 Alice
no 2042303037888785483 em o
País 3157451138509946042 País
das 1295000510

In [30]:
tokens = [str(token) for token in doc if token.is_stop == False]
tokens = [token for token in tokens 
          if re.search(r'^[A-Za-záàâãéèêíóôõúçÁÀÂÃÉÈÍÓÔÕÚÇ\- ]+$', token)]

for token in tokens:
    entry = lex[lex['palavra'] == token]
    print(f"{token} : {entry['freq'].values}")

funciona : [3430]
Origem : [335]
coisas : [18954]
Coelho : [4323]
Branco : [6839]
personagem : [5280]
livro : [23518]
Alice : [1159]
País : [1752]
Maravilhas : [66]
inglês : [7213]
Lewis : [491]
Carroll : [86]
entrava : [1423]
pânico : [920]
olhava : [1449]
relógio : [1489]
desespero : [2225]
símbolo : [1937]
objetos : [3852]
recém-incorporados : []
cotidiano : [1568]
estavam : [14391]
fazendo : [11837]
cabeça : [17153]
bisavós : [21]
século : [11409]
anos : [107768]
impossível : [5339]
pensar : [6999]
existência : [7485]
tique-taque : []
assinala : [205]
hora : [19904]
acordar : [790]
dormir : [2924]
comer : [2830]
atividades : [10542]
humanas : [1535]
necessidade : [10713]
maneira : [12450]
única : [11280]
medir : [872]
deu : [19733]
origem : [7305]
primeiros : [12577]
relógios : [425]
sol : [7383]
Egito : [1280]
há : [74317]
cerca : [26661]
anos : [107768]
aparelhos : [1776]
mecânicos : [384]
ponteiros : [128]
surgiram : [1375]
China : [4790]
século : [11409]
Europa : [10482]
começa

In [ ]:

#analyze_and_create_output_csv('dataset', 'output.csv')

In [ ]:
df = create_dataframe('dataset_caed')

analysis_dict_list = []
nlp = spacy.load('pt_core_news_sm')
nlp.add_pipe('textdescriptives/all')

row = df.iloc[0]

doc = nlp(row['text'])
text_info = td.extract_df(doc).iloc[0].to_dict()
text_info = {**text_info, **row}
analysis_dict_list.append(text_info)  # access some of the values

analysis_df = pd.DataFrame(analysis_dict_list)

In [ ]:
analysis_df.dropna()
analysis_df

In [ ]:
pd.DataFrame(text_info).to_dict()

In [ ]:
a_df = analyze_text_append_result(df, 'text')

In [ ]:
a_df